# Урок 4

В этом уроке мы перейдем от DTW к распознованию по эталонному словарю. 

Эталон в графе теперь будет задаваться как последовательность фонем из словаря. Словарь для да/нет (daNetDict.txt) выглядит следующим образом:

da: SIL d aa SIL<br>
net: SIL n ee t SIL<br>

Словарь для цифр (digitDict.txt):

0: SIL n ay ll SIL<br>
1: SIL a dd ii n SIL<br>
2: SIL d v aa SIL<br>
3: SIL t rr ii SIL<br>
4: SIL ch i t yy rr e SIL<br>
5: SIL pp aa tt SIL<br>
6: SIL sh ee s tt SIL<br>
7: SIL ss ee mm SIL<br>
8: SIL v oo ss ae mm SIL<br>
9: SIL dd ee vv ae tt SIL<br>

SIL здесь – это фонема паузы. 

Каждой фонеме будет сопоставлен наиболее типичный для нее вектор признаков. В данном случае мы будем использовать среднее значение вектора признаков фонем, взятых из обучающего набора. Будем назвать это акустической моделью, которая уже храниться в файле outAM.

Здесь граф уже строится на основе словаря. Узлы теперь представляют собой отдельные фонемы с переходом только в себя и следующий узел (переходов через несколько узлов уже не будет, так как пропуск фонемы в слове нежелателен). Набор используемых фонем и их вектора признаков (AMs - экземпляр класса AcoModelSet, который представляет собой словарь: ключ - фонема, который указывает на акустическую модель этой фонемы - экземпляр класса AcoModel) определяются акустической моделью, загружаемой с помощью AcoModels из outAM. Загрузить акустическую модель можно следующим способом: AMs = AcoModels.loadAcoModelSet(file_AMs). Схема такого графа для да/нет (для цифр по аналогии) представлена ниже:

<img src="graph_4.jpg">

<b>Задание 1:</b> 
- Написать функцию для чтения словаря.
- Разобраться с описанием класса AcoModel из файла AcoModels.py
- Реализовать граф на основе словаря и загруженной акустической модели AMs.

In [1]:
import numpy as np
import FtrFile
import AcoModels


class State:
    # model - экземляр класса AcoModel
    def __init__(self, model, idx): 
        self.model = model
        self.word = None
        self.isFinal = False
        self.nextStates = []
        self.idx = idx
        self.bestToken = None
        self.currentWord = None


# rxfilename - файл словаря
def load_graph(rxfilename, AMs):
    
    startState = State(None, 0)
    graph = [startState, ]
    stateIdx = 1
    #--------------------TODO--------------------------------------------
    
    #---------------------------------------------------------------------
    return graph

Проверка графа:

In [2]:
def check_graph(graph):
    assert len(graph) > 0, "graph is empty."
    assert graph[0].model is None \
        and graph[0].word is None \
        and not graph[0].isFinal, "broken start state in graph."
    idx = 0
    for state in graph:
        assert state.idx == idx
        idx += 1
        assert (state.isFinal and state.word is not None) \
            or (not state.isFinal and state.word is None)


def print_graph(graph):
    with open('graph.txt', 'w') as fn:
        np.set_printoptions(formatter={'float': '{: 0.1f}'.format})
        for state in graph:
            nextStatesIdxs = [s.idx for s in state.nextStates]
            if state.idx == 0:
                model_name = 'None'
            else:
                model_name = state.model.name    
            fn.write("State: idx={} word={} isFinal={} nextStatesIdxs={} model={} \n".format(
                state.idx, state.word, state.isFinal, nextStatesIdxs, model_name))
    print("*** SEE graph.txt ***")
    print("*** END DEBUG. GRAPH ***")


file_etalons = "daNetDict.txt" 
#file_etalons = "digitDict.txt"
file_AMs = "outAM"
# загрузка акустической модели из уже готового файла outAM:
AMs = AcoModels.loadAcoModelSet(file_AMs)
    
graph = load_graph(file_etalons, AMs)
                   
check_graph(graph)
# Сохранить граф в читабельном виде в файл graph.txt:
print_graph(graph)

*** SEE graph.txt ***
*** END DEBUG. GRAPH ***


Теперь дистанция будет считаться как евклидово расстояние признаков кадра записи от признаков фонемы в эталоне с помощью метода dist класса акустической модели.

<b>Задание 2:</b> Вычислить дистанцию токена с помощью метода класса AcoModel.

In [6]:
class Token:
    def __init__(self, state, dist=0.0, sentence=""):
        self.state = state
        self.dist = dist
        self.sentence = sentence
        self.alive = True
        
        
def findBest(Tokens):
    minDist = None
    bestToken = 0
    for token in Tokens:
        if token.alive:
            if not minDist:
                minDist = token.dist
                bestToken = token
            elif token.dist <= minDist:
                minDist = token.dist
                bestToken = token
    return bestToken


def beamPurnning(nextTokens):
    thr_common = 150        
    bestToken = findBest(nextTokens)
    bestDist = bestToken.dist
    for i in range(len(nextTokens)):
        if nextTokens[i].alive and nextTokens[i].dist > bestDist + thr_common:
            nextTokens[i].alive = False
    return nextTokens, bestDist


def statePrunning(nextTokens):
    for i in range(len(nextTokens)):
        state_index = nextTokens[i].state.idx
        if not graph[state_index].bestToken:
            graph[state_index].bestToken = nextTokens[i]
        else:
            if nextTokens[i].dist <= graph[state_index].bestToken.dist:
                graph[state_index].bestToken.alive = False
                graph[state_index].bestToken = nextTokens[i]
            else:
                nextTokens[i].alive = False
    for token in nextTokens:                           # cleaning the bestTokens
        index = token.state.idx                        #
        if graph[index].bestToken:                     #
            graph[index].bestToken = None
    
    return nextTokens


def recognize(rec_results, features, graph):

    startTime = time.time()
    print("-" * 20)
    startState = graph[0]
    activeTokens = [Token(startState), ]
    nextTokens = []

    for frame in range(features.nSamples):
        ftrCurrentFrameRecord = features.readvec()
        for token in activeTokens:
            if token.alive:
                for transitionState in token.state.nextStates:
                    newToken = Token(transitionState, token.dist, token.sentence)
                    #----------------------TODO---------------------------------------
                    #newToken.dist += 
                    #-----------------------------------------------------------------
                    nextTokens.append(newToken)
        nextTokens = statePrunning(nextTokens)               
        nextTokens,bestDist = beamPurnning(nextTokens)        

        activeTokens = nextTokens
        nextTokens = []                                    

    finalTokens = []
    for token in activeTokens:
        if token.state.isFinal and token.alive:
            finalTokens.append(token)

    if len(finalTokens) != 0:
        winToken = findBest(finalTokens)
    else:
        winToken = findBest(activeTokens)
        winToken.state.word = winToken.state.currentWord

    print("result: {} ==> {}".format(filename, winToken.state.word))
    endTime = time.time()
    print("time: {} sec".format(round(endTime - startTime, 2)))

    etalons_word = filename.split('_')[0]
    records_word = winToken.state.word.split('_')[0]
    rec_results.append(etalons_word == records_word)

    return frame

Здесь поддерживается распознавание да/нет и одиночных цифр. Для выбора первого варианта в коде ниже нужно указать <br> "base = 1", а для второго – любое другое значение отличное от 1.

In [10]:
    import time
    
    base = 1
    if base == 1:
        file_etalons = "daNetDict.txt"
        records = "ark,t:records_daNet_mfcc.txtftr"
        #records = "ark,t:DaNetStreet_mfcc.txtftr"
    else:
        file_etalons = "digitDict.txt"
        records = "ark,t:records_digit_mfcc.txtftr"
        #records = "ark,t:DigitStreet_mfcc.txtftr"

    file_AMs = "outAM"

    rec_results = []
    s_time = time.time()
    numbFrame = 0

    # загрузка акустической модели из уже готового файла outAM:
    AMs = AcoModels.loadAcoModelSet(file_AMs)
    
    graph = load_graph(file_etalons, AMs)
    for filename, features in FtrFile.FtrDirectoryReader(records):
        frame = recognize(rec_results, features, graph)
        numbFrame += frame
    print("-" * 20)
    print("WER is: {}".format(round(1 - sum(rec_results) / len(rec_results), 3)))

    time = time.time() - s_time
    minut = int(time / 60)
    second = time - minut * 60
    print("Total time: {} min {} sec".format(minut, round(second,2)))
    rtf = time / (numbFrame * 0.01)
    print("RTF is: {}".format(round(rtf,3)))

--------------------
result: da_06 ==> da
time: 0.04 sec
--------------------
result: da_07 ==> da
time: 0.04 sec
--------------------
result: da_08 ==> da
time: 0.04 sec
--------------------
result: da_09 ==> da
time: 0.04 sec
--------------------
result: da_10 ==> da
time: 0.04 sec
--------------------
result: da_11 ==> da
time: 0.06 sec
--------------------
result: da_12 ==> da
time: 0.04 sec
--------------------
result: da_13 ==> da
time: 0.06 sec
--------------------
result: da_14 ==> da
time: 0.05 sec
--------------------
result: da_15 ==> da
time: 0.06 sec
--------------------
result: da_16 ==> da
time: 0.05 sec
--------------------
result: da_17 ==> da
time: 0.05 sec
--------------------
result: da_18 ==> da
time: 0.07 sec
--------------------
result: da_19 ==> da
time: 0.05 sec
--------------------
result: net_06 ==> net
time: 0.04 sec
--------------------
result: net_07 ==> net
time: 0.06 sec
--------------------
result: net_08 ==> net
time: 0.06 sec
--------------------
resu

Как вы уже, наверное, заметили, такой вариант распознавания работает значительно быстрее расcмотренного DTW, да и WER теперь для да/нет вообще равен нулю.